In [2]:
import tensorflow as tf

In [2]:
from keras import Model
from keras.layers import Embedding, LSTM, Dropout
from keras.optimizers import Adam

In [ ]:
NUM_WORDS = 3000 #본문 기사 내 가장 많이 사용된 3000단어
VECTOR_SIZE_NOT_DETERMINED_YET = 100
MAX_SUMMARIZATION_LENGTH_NOT_DETERMINED_YET = 500

In [ ]:
class Seq2Seq():
    def __init__(self):
        self.VECTOR_SIZE_NOT_DETERMINED_YET = 100
        self.MAX_SUMMARIZATION_LENGTH_NOT_DETERMINED_YET = 500
        
        self.sos = sos
        self.eos = eos
        
        self.optimizer = Adam()
        
        encoder_inputs = Input(shape=(self.VECTOR_SIZE_NOT_DETERMINED_YET,))
        decoder_inputs = Input(shape=(self.MAX_SUMMARIZATION_LENGTH_NOT_DETERMINED_YET,))
        
        self.enc = self.build_encoder()
        hc_list = self.enc(encoder_inputs)
        
        self.dec = self.build_decoder(hc_list)
        summary_softmaxed = self.dec(decoder_inputs)
        
        #ㅁ//summary_softmaxed는 전체 3000개 freq 단어(3000)에 대한 softmax값..?으로해야하는건지
        #ㅁ//Or 데이터 내 전체 단어에 대한 softmax값으로 써야할 지 모르겠음
        
        self.seq2seq = Model(inputs=encoder_inputs, outputs=summary_softmaxed)
        self.seq2seq.compile(loss='cross_categorical_entropy', optimizer=self.optimizer)
        
        
    def build_encoder(self):
        def lstm(inputs, hs):
            output,h,c = LSTM(hs, return_state=True)(inputs) #return only last h, c
#             e = Dropout(0.3)(e)
            return output, h, c
        
        #(encoder input) already embedded from koBERT   
        encoder_inputs = Input(shape=(self.VECTOR_SIZE_NOT_DETERMINED_YET,))
        _, h, c = lstm(encoder_inputs, 512) #Discard encoder outputs
        
        return Model(encoder_inputs, [h,c])
    
    
    def build_decoder(self, hc_list):
        def lstm(inputs, hs, initial_state):
            output,h,c = LSTM(hs, return_sequences=True, return_state=True)(inputs, initial_state=initial_state) #returns all -> h1,...hn / c1,...cn
#             e = Dropout(0.3)(e)
            return output, h, c

        def fc(n_h_layers, inputs, hn):
            for _ in range(n_h_layers):
                d = Dense(hn, activation='tanh')(inputs)
            if n_h_layers==0: 
                d = inputs
            output = Dense(hn, activation='softmax')(d)
            #모든 3000개 단어에 대한 확률값 (해당 위치에서의)
            
        decoder_inputs = Input(shape=(self.MAX_SUMMARIZATION_LENGTH_NOT_DETERMINED_YET,))
        
        outputs, _, _ = lstm(decoder_inputs, 512, hc_list) #Discard h,c
        
        outputs_softmaxed = fc(1, outputs, NUM_WORDS) #ㅁ//NUM_WORDS가 맞는지?
        return Model(decoder_inputs, outputs_softmaxed)      
            
    def train(self, epochs, batch_size=1):
        

In [ ]:
class Encoder(Model):
    def __init__(self):
        super(Encoder, self).__init__() 
        self.lstm = LSTM(512, return_state=True) 
        
    def __call__(self, x, training=False, mask=None):
        
        _, h, c = self.lstm(x) #Discard encoder outputs.
        return h, c

In [ ]:
class Decoder(Model):
    def __init__(self):
        super(Decoder, self).__init__()
        self.lstm = LSTM(512, return_sequences=True, return_state=True) #h1,...hn / c1,...cn까지 모두 출력받음
        
        self.dense1 = Dense(NUM_WORDS, activation='tanh')
        self.dense2 = Dense(NUM_WORDS, activation='softmax') #to make output
        #모든 3000개 단어에 대한 확률값 (해당 위치에서의)
        
    def __call__(self, inputs, training=False, mask=None):
        #inputs given as (decoderinput, h, c)
        #h, c for inital state of decoder lstm
        #x(input) already embedded from koBERT
        x, h, c = inputs
        
        outputs, _, _ = self.lstm(x, initial_state=[h,c]) #Discard h,c
        
        outputs_1 = self.dense1(outputs)
        outputs_2 = self.dense2(outputs_1) 
        return outputs_2

In [12]:
class seq2seq(Model):
    def __init__(self, sos, eos):
        super(seq2seq, self).__init__()
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos
        self.eos = eos
        
    def __call__(self, inputs, training=False):
        if training:
            x, inp = tuple(inputs) #x for encoder inputs. inp for decoder inputs
            
            h,c = self.enc(x)
            
            y = self.dec((inp,h,c)) 
            
            return y
        
        else: #no inputs for decoder
            '''shifted output for inference'''
            x = inputs
            h,c = self.enc(x)
            
            #decoder단에 가장 먼저 sos삽입
            inp = tf.convert_to_tensor(self.sos)
            inp = tf.reshape(inp, (1,1)) #input길이가 하나밖에 없음
            
            seq = tf.TensorArray(tf.int32, 64) #정답array - 최대 64길이까지 출력으로 받는다(summarization을)
            
            #shifted output
            for _ in tf.range(64): #tf.range better for keras Models
                y = self.dec([inp,h,c]) #input길이 하나밖에 없기때문에 한개값 출력받을 것임
                
                #shifted output
                inp = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32) #for next input
                inp = tf.reshape(inp, (1,1))
                
                seq = seq.write(_,inp)
                
                if inp == self.eos:
                    break
            
            return tf.reshape(seq.stack(), (1,64)) # stack은 그동안 TensorArray로 받은 값을 쌓아주는 작업을 한다.   
            
            